In [1]:
import re
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import LabelEncoder


In [2]:
# Чтение данных из файла
with open('vin_ford_train.txt', 'r') as file:
    data = file.read()

In [3]:
# Паттерн для поиска VIN-кода и цены
pattern = re.compile(r'\[([A-Z0-9]{17})\:(\d+)\]')
# Находим все соответствия паттерну в файле
matches = pattern.findall(data)

In [4]:
# Создаем список кортежей, содержащих VIN-коды и цены
data_list = [(vin, int(price)) for vin, price in matches]

# Создаем датафрейм из списка
df = pd.DataFrame(data_list, columns=['VIN_code', 'price'])

# Выводим датафрейм
display(df.head())

,VIN_code,price
0,2FMDK3JC4BBA41556,12500
1,3FA6P0H75ER208976,14500
2,3FAHP08Z17R268380,1300
3,1FMCU0GX3DUC59421,15200
4,1FAHP2E89DG119368,11600


In [5]:
# Проверим валидность VIN-кодов. 
# При проверке используем логику для проверки контрольной суммы VIN-кода по алгоритму Luhn.

# Определим функцию для проверки валидности VIN-кода
def is_valid_vin(vin):
    if len(vin) != 17:
        return False
    if not re.match(r'^[A-HJ-NPR-Z0-9]+$', vin):
        return False
    
    # Проверка контрольной суммы
    transliteration = "0123456789.ABCDEFGH..JKLMN.P.R..STUVWXYZ"
    weights = [8, 7, 6, 5, 4, 3, 2, 10, 0, 9, 8, 7, 6, 5, 4, 3, 2]
    checksum = 0
    for i in range(17):
        if transliteration.find(vin[i]) == -1:
            return False
        checksum += weights[i] * (transliteration.find(vin[i]) % 10)
    if vin[8].isdigit():
        expected = int(vin[8])
    elif vin[8] == 'X':
        expected = 10
    else:
        return False
    return (checksum % 11) == expected

# Применяем функцию к столбцу 'VIN_code' и создаем маску для невалидных VIN-кодов
invalid_vin_mask = ~df['VIN_code'].apply(is_valid_vin)

# Создаем датафрейм с невалидными VIN-кодами
invalid_vin_df = df[invalid_vin_mask]

# Удаляем невалидные VIN-коды из исходного датафрейма
df = df[~invalid_vin_mask]

# Выводим датафрейм с невалидными VIN-кодами
print("Невалидные VIN-коды:")
print(invalid_vin_df)

Невалидные VIN-коды:
               VIN_code  price
1309  1FM5K8D83DGB9100X  26600


In [6]:
# Разделяем VIN код на составляющие
df['WMI_country'] = df['VIN_code'].str.slice(0, 1) # Страна производства

df['WMI_manufacturer'] = df['VIN_code'].str.slice(1, 3) # Код производителя

df['VDS_model'] = df['VIN_code'].str.slice(3, 6) # Модель автомобиля.

df['VDS_body_type'] = df['VIN_code'].str.slice(6, 7) # Тип кузова автомобиля

df['VDS_engine_type'] = df['VIN_code'].str.slice(7, 8) # Тип двигателя

df['VDS_check_digit'] = df['VIN_code'].str.slice(8, 9) # Контрольная цифра, используемая для проверки валидности VIN кода

df['year_of_manufacture'] = df['VIN_code'].str[9] # Год производства

df['VIS_serial_number'] = df['VIN_code'].str.slice(10) # Заводской номер автомобиля

df.drop(['VIN_code'], axis=1, inplace=True)

display(df.head())

,price,WMI_country,WMI_manufacturer,VDS_model,VDS_body_type,VDS_engine_type,VDS_check_digit,year_of_manufacture,VIS_serial_number
0,12500,2,FM,DK3,J,C,4,B,BA41556
1,14500,3,FA,6P0,H,7,5,E,R208976
2,1300,3,FA,HP0,8,Z,1,7,R268380
3,15200,1,FM,CU0,G,X,3,D,UC59421
4,11600,1,FA,HP2,E,8,9,D,G119368


In [7]:
# Преобразование категориальных признаков с помощью Label Encoding
label_encoder = LabelEncoder()
categorical_features = ['WMI_country', 'WMI_manufacturer', 'VDS_model', 
                        'VDS_body_type', 'VDS_engine_type', 
                        'VDS_check_digit', 'year_of_manufacture',
                        'VIS_serial_number']
for feature in categorical_features:
    df[feature] = label_encoder.fit_transform(df[feature])

In [8]:
df.head()

,price,WMI_country,WMI_manufacturer,VDS_model,VDS_body_type,VDS_engine_type,VDS_check_digit,year_of_manufacture,VIS_serial_number
0,12500,1,5,32,18,11,4,10,5289
1,14500,2,2,3,17,6,5,13,55722
2,1300,2,2,82,8,31,1,6,58118
3,15200,0,5,18,16,29,3,12,67866
4,11600,0,2,83,14,7,9,12,23755


In [9]:
# Разделяем данные на обучающий и тестовый наборы
X = df.drop(['price'], axis=1)  # Входные признаки
y = df['price']  # Целевая переменная

In [10]:
# Разделение данных на обучающий и тестовый наборы
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12345)

# Проверка размеров наборов данных
print("Размер обучающего набора данных:", X_train.shape[0])
print("Размер тестового набора данных:", X_test.shape[0])

Размер обучающего набора данных: 57507
Размер тестового набора данных: 14377


In [11]:
# Хорошим практическим подходом является масштабирование признаков 
# после разделения данных на обучающую и тестовую выборки.
# Масштабируем признаки

# Создание объекта StandardScaler
scaler = StandardScaler()

# Выбор признаков для масштабирования
features_to_scale = ['WMI_country', 'WMI_manufacturer', 'VDS_model', 'VDS_body_type', 'VDS_engine_type', 'VDS_check_digit', 'year_of_manufacture', 'VIS_serial_number']

# Масштабирование выбранных признаков
X_train_scaled = X_train.copy()  
X_train_scaled[features_to_scale] = scaler.fit_transform(X_train_scaled[features_to_scale])

X_test_scaled = X_test.copy()  
X_test_scaled[features_to_scale] = scaler.fit_transform(X_test_scaled[features_to_scale])

In [12]:
# Инициализация модели
model = CatBoostRegressor(iterations=1000, learning_rate=0.1, depth=6, verbose=False)

# Обучение модели
model.fit(X_train_scaled, y_train)
model.save_model('catboost_model.cbm')


In [13]:
# Предсказание на тестовом наборе данных
y_pred = model.predict(X_test_scaled)

In [14]:
# Вычисление метрик

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100 # Вычисление MAPE
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mse) 
mape = mean_absolute_percentage_error(y_test, y_pred)

# Организация метрик в таблицу
metrics_df = pd.DataFrame({
    'Mean Absolute Percentage Error': mape,
    'Mean Squared Error': [mse],
    'Mean Absolute Error': [mae],
    'R-squared': [r2],
    'Root Mean Squared Error': [rmse] 
})

display(metrics_df)

,Mean Absolute Percentage Error,Mean Squared Error,Mean Absolute Error,R-squared,Root Mean Squared Error
0,20.32626,8.850736e+06,2025.148004,0.853671,2975.018635


In [15]:
# Получение важности признаков
feature_importance = model.feature_importances_

# Создание DataFrame с важностью признаков
importance_df = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': feature_importance
})

# Сортировка признаков по важности
importance_df = importance_df.sort_values(by='Importance', ascending=False)

print(importance_df)

               Feature  Importance
3        VDS_body_type   24.939754
7    VIS_serial_number   20.746526
1     WMI_manufacturer   16.397202
6  year_of_manufacture   15.304042
2            VDS_model   13.932504
4      VDS_engine_type    8.166250
0          WMI_country    0.283155
5      VDS_check_digit    0.230567
